In [35]:
from pandas import Series 
import pandas as pd 
import numpy as np
from IPython.display import HTML

#data = pd.read_csv('1.C.1.csv', sep=';', index_col=[0, 1, 2, 3])
#data.sort_index(inplace=True)

#idx = pd.IndexSlice
#tmp = data.loc[idx[:, 'Psychologie', :, 2018], :]

#print(tmp.index.get_level_values('university').unique())
#display(data.loc['FWF'].xs(2018, level='year'))
#display(data.xs(2018, level='year'))

class Dimension:
    @classmethod
    def create(cls, dataframe, dimensions):
        dims = dict()
        for dim in dimensions:
            assert dim in dataframe.index.names, "There is no index '" + dim + "'"
            cats = dataframe.index.get_level_values(dim).unique()
            dims[dim] = Dimension(dim, cats.values)
        return dims
    
    def __init__(self, name, categories):
        self.__name = name
        self.__categories = categories   
    @property
    def name (self):
        return self.__name
    @property
    def categories(self):
        return self.__categories  
    def __str__(self):
        return self.name
        
class Datasheet:
    def __init__(self, filename, dimensions, timepoint_name, university_name, value_name):
        dataframe = pd.read_csv(filename, sep=';', index_col=list(range(len(dimensions)+2)))        
        dataframe.columns = [value_name]
        dataframe.sort_index(inplace=True)

        assert timepoint_name in dataframe.index.names, "There is no index '" + timepoint_name + "'"
        assert university_name in dataframe.index.names, "There is no index '" + university_name + "'"
        assert value_name in dataframe, "There is no colum with valid data called '" + value_name + "'"
        
        self.__dataframe = dataframe
        self.__timepoint_idx = timepoint_name
        self.__university_idx = university_name
        self.__value_column = value_name       
        self.__dimensions = Dimension.create(dataframe, dimensions + [university_name, timepoint_name])
        self.__timepoints = dataframe.index.get_level_values(timepoint_name).unique()
        self.__universities = dataframe.index.get_level_values(university_name).unique()
        self.__recent_timepoint = self.__timepoints.max()
                
    @property
    def value_column(self):
        return self.__value_column
    @property
    def university_idx(self):
        return self.__university_idx
    @property
    def timepoint_idx(self):
        return self.__timepoint_idx
    @property
    def dimensions(self):
        return self.__dimensions
    @property
    def most_recent(self):
        return self.__recent_timepoint
    @property
    def universities(self):
        return self.__universities
    
    def only(self, conditions):
        df = self.__dataframe
        if conditions is None:
            return df
        slices = list()
        for dim in self.dimensions.keys():
            if self.dimensions[dim].name in conditions.keys():
                slices.append(conditions[self.dimensions[dim].name])
            else:
                slices.append(slice(None))
        return df.loc[tuple(slices), :]
        
    def analyze(self, description, about, filtering=None):
        return Analysis(self, description, self.dimensions[about], self.only(filtering) \
                            .sum(level = [self.dimensions[about].name, self.university_idx]))


class Analysis:
    def __init__(self, sheet, description, about, table):
        self.__sheet = sheet
        self.__table = table.unstack(about.name, fill_value=0)
        self.__about = about
        self.__desc = description
        
        self.__sums = table.sum(axis='columns')
        self.__sums.name = sheet.value_column
        self.__totals = table.sum(level=[about.name])[sheet.value_column]
        self.__grand_total = table.sum()
        self.__grand_total.name = 'grand total'
        
        assert self.__grand_total[sheet.value_column] == self.__sums.sum()
        
        self.__structure = Structure(self)
        self.__ranking = Ranking(self)

               
    @property
    def sheet(self):
        return self.__sheet
    @property
    def table(self):
        return self.__table
    @property
    def horizontal_sums(self):
        return self.__sums
    @property
    def sums(self):
        return self.horizontal_sums
    @property
    def vertical_sums(self):
        return self.__totals
    @property
    def totals(self):
        return self.vertical_sums
    @property
    def about(self):
        return self.__about
    @property
    def description(self):
        return self.__desc
    @property
    def grand_total(self):
        return self.__grand_total
    @property
    def structure(self):
        return self.__structure
    @property
    def ranking(self):
        return self.__ranking
    
    def values(self, university):
        tmp = self.table.loc[university]
        tmp.name = self.sheet.value_column
        return tmp
    

class Structure:
    def __init__(self, analysis):
        self.__analysis = analysis
        tmp = analysis.sums.sum(level=[analysis.sheet.university_idx])
        self.__table = analysis.table.divide(tmp, axis=0)
        
    @property
    def table(self):
        return self.__table
    @property
    def analysis(self):
        return self.__analysis
    
    def most_important_to(self, university, cutoff=0.9):
        u = self.table.loc[university].sort_values(ascending=False)
        u.name = 'relevance'
        cumsum = u.cumsum()
        if cumsum.iloc[0] > cutoff:
            return u[0:1]
        else:
            return u[u.cumsum() < cutoff]

    def of(self, university):
        tmp = self.table.loc[university].sort_values(ascending=False)
        tmp.name = 'relevance'
        return tmp
    
    def differences_to(self, university, cutoff=0.9):
        u = self.most_important_to(university, cutoff)
        difference_matrix =  (u-self.table).abs() / u.count()
        difference_matrix.replace([np.inf, -np.inf], np.nan, inplace=True)     
        #difference_matrix.replace(np.nan, 0, inplace=True)
        return difference_matrix
        
    def most_similar_to(self, university, cutoff=0.9):
        difference_matrix = self.differences_to(university, cutoff)
        result = difference_matrix.sum(axis='columns')
        result = result.sort_values()[1:]
        result.name = 'differences'
        return result
    
    
class Ranking:
    def __init__(self, analysis):
        self.__analysis = analysis
        self.__rankings = analysis.table.rank(axis='rows', method='max', ascending=False)
        
        tmp = analysis.horizontal_sums.sum(level=self.analysis.sheet.university_idx)
        self.__total_ranking = tmp.rank(method='max', ascending=False).sort_values()
        self.__total_ranking.name = 'rank'
        
    @property
    def analysis(self):
        return self.__analysis
    @property
    def table(self):
        return self.__rankings
    
    def of(self, university, category=None):
        categories = self.__analysis.about.categories
        retval = None
        if category == None:
            retval = self.__total_ranking.loc[university]
        else:
            if category in categories:
                retval = self.__rankings[self.analysis.sheet.value_column, category][university]
            else:
                retval = self.__rankings.loc[university].astype(int)
        retval.name = 'ranking'
        return retval

    def better_than(self, university, category=None):
        if category == None:
            return self.__total_ranking[self.__total_ranking < self.__total_ranking[university]]
        else:
            tmp = self.__rankings[self.analysis.sheet.value_column, category].sort_values()
            return tmp[tmp[university] > tmp]
        
    def best(self, category):
        tmp = self.__rankings[self.analysis.sheet.value_column, category].sort_values()[0:1]
        return tmp
        
def highlight_places(column, series):
    if series[column] == 1:
        return ['background-color: lightgreen']*5
    elif series[column] == 2:
        return ['background-color: yellow']*5
    else:
        return ['background-color: white']*5
            
class University:
    def __init__(self, name):
        self.__name = name
        self.__analyses = list()
        
    @property
    def name(self):
        return self.__name
    
    def add_analysis(self, analysis):
        self.__analyses.append(analysis)
        
    def describe(self):
        display(HTML("<h3>" + self.name + "</h3>"))
        for a in self.__analyses:
            display(HTML("<h4>" + a.description + "-Struktur</h4>"))
            df = a.structure.most_important_to(self.name, 2/3)
            df = df[df > 0].dropna().sort_values(ascending=False).to_frame()

            df = df.join(a.ranking.of(self.name, 'all')) \
                   .join(a.values(self.name))
            
            best = pd.Series()
            better = pd.Series()
            best.name = 'best'
            better.name = 'better'
            for cat in df.index.get_level_values(level=a.about.name):
                best_uni = a.ranking.best(cat).index.values[0]
                if best_uni == self.name:
                    best_uni = '(diese)'
                else: 
                    best_uni += ' (' + '{0:,.1f} Mio. €'.format(a.values(best_uni)[a.sheet.value_column, cat]/1000000) + ')'
                    
                better_unis = ', '.join(a.ranking.better_than(self.name, cat).index.values)
                if better_unis == '':
                    better_unis = '(keine)'
                    
                best.at[cat] = best_uni
                better.at[cat] = better_unis
                
            df = df.join(best, on=a.about.name).join(better, on=a.about.name)
            df = df.droplevel(0)
            
            df.index.name = 'WZW'
            df.columns = ['Intern', 'Ranking', 'Erlöse', 'Beste Universität', 'Bessere Universitäten']
            
            style = df.style.format({'Intern': '{:.2%}', 'Ranking': '{:}. Platz',
                                     'Erlöse': '{:,} €'})
                        
            style = style.apply(lambda s: highlight_places('Ranking', s), axis=1)
            display(style)
            
            display(HTML('<h4>Ähnliche Universitäten aufgrund der ' + a.description + '-Struktur</h3>'))
            similars = a.structure.most_similar_to(self.name, 2/3)
            display(HTML('<p>' + ', '.join(similars.index.values[0:4]) + '</p>'))
                

ds = Datasheet('1.C.1.csv', ['funder', 'fos'], 'year', 'university', 'funding')
analysis = ds.analyze("Drittmittel", "funder", {'year': ds.most_recent})

universities = list()

for uni_name in ds.universities:
    u = University(uni_name)
    u.add_analysis(analysis)
    universities.append(u)
    
for u in universities:
    u.describe()
    
    

,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
Unternehmen,30.33%,6. Platz,"14,549,837 €",Medizinische Universität Graz (32.4 Mio. €),"Medizinische Universität Graz, Technische Universität Graz, Technische Universität Wien, Medizinische Universität Wien, Medizinische Universität Innsbruck"
FWF,17.04%,7. Platz,"8,173,203 €",Universität Wien (41.1 Mio. €),"Universität Wien, Technische Universität Wien, Medizinische Universität Wien, Universität Innsbruck, Universität Graz, Medizinische Universität Innsbruck"
EU,15.67%,6. Platz,"7,516,953 €",Technische Universität Wien (14.7 Mio. €),"Technische Universität Wien, Universität Wien, Technische Universität Graz, Universität Innsbruck, Medizinische Universität Wien"


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
Unternehmen,21.30%,13. Platz,"1,606,003 €",Medizinische Universität Graz (32.4 Mio. €),"Medizinische Universität Graz, Technische Universität Graz, Technische Universität Wien, Medizinische Universität Wien, Medizinische Universität Innsbruck, Universität für Bodenkultur Wien, Montanuniversität Leoben, Universität Linz, Universität Innsbruck, Universität Wien, Veterinärmedizinische Universität Wien, Universität Graz"
FWF,18.28%,14. Platz,"1,378,473 €",Universität Wien (41.1 Mio. €),"Universität Wien, Technische Universität Wien, Medizinische Universität Wien, Universität Innsbruck, Universität Graz, Medizinische Universität Innsbruck, Universität für Bodenkultur Wien, Universität Linz, Universität Salzburg, Technische Universität Graz, Veterinärmedizinische Universität Wien, Medizinische Universität Graz, Wirtschaftsuniversität Wien"
EU,13.58%,14. Platz,"1,024,247 €",Technische Universität Wien (14.7 Mio. €),"Technische Universität Wien, Universität Wien, Technische Universität Graz, Universität Innsbruck, Medizinische Universität Wien, Universität für Bodenkultur Wien, Montanuniversität Leoben, Medizinische Universität Innsbruck, Universität Salzburg, Universität Graz, Universität Linz, Medizinische Universität Graz, Wirtschaftsuniversität Wien"
Bund (Ministerien),12.74%,9. Platz,"960,387 €",Medizinische Universität Innsbruck (3.7 Mio. €),"Medizinische Universität Innsbruck, Universität Wien, Universität für Bodenkultur Wien, Montanuniversität Leoben, Technische Universität Wien, Medizinische Universität Wien, Technische Universität Graz, Universität Salzburg"


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
FWF,48.27%,1. Platz,"41,065,705 €",(diese),(keine)
EU,16.02%,2. Platz,"13,633,780 €",Technische Universität Wien (14.7 Mio. €),Technische Universität Wien


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
Unternehmen,25.31%,3. Platz,"22,101,493 €",Medizinische Universität Graz (32.4 Mio. €),"Medizinische Universität Graz, Technische Universität Graz"
FWF,22.52%,2. Platz,"19,662,614 €",Universität Wien (41.1 Mio. €),Universität Wien
FFG,18.69%,2. Platz,"16,317,706 €",Technische Universität Graz (19.0 Mio. €),Technische Universität Graz


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
FWF,33.00%,4. Platz,"16,317,638 €",Universität Wien (41.1 Mio. €),"Universität Wien, Technische Universität Wien, Medizinische Universität Wien"
EU,17.28%,4. Platz,"8,544,360 €",Technische Universität Wien (14.7 Mio. €),"Technische Universität Wien, Universität Wien, Technische Universität Graz"
Länder (inkl. deren Stiftungen und Einrichtungen),11.12%,2. Platz,"5,495,536 €",Universität Wien (5.6 Mio. €),Universität Wien


...

,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
FWF,26.56%,8. Platz,"7,769,848 €",Universität Wien (41.1 Mio. €),"Universität Wien, Technische Universität Wien, Medizinische Universität Wien, Universität Innsbruck, Universität Graz, Medizinische Universität Innsbruck, Universität für Bodenkultur Wien"
FFG,26.09%,4. Platz,"7,632,744 €",Technische Universität Graz (19.0 Mio. €),"Technische Universität Graz, Technische Universität Wien, Montanuniversität Leoben"


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
FWF,40.14%,19. Platz,"465,349 €",Universität Wien (41.1 Mio. €),"Universität Wien, Technische Universität Wien, Medizinische Universität Wien, Universität Innsbruck, Universität Graz, Medizinische Universität Innsbruck, Universität für Bodenkultur Wien, Universität Linz, Universität Salzburg, Technische Universität Graz, Veterinärmedizinische Universität Wien, Medizinische Universität Graz, Wirtschaftsuniversität Wien, Universität Klagenfurt, Montanuniversität Leoben, Universität für angewandte Kunst Wien, Akademie der bildenden Künste Wien, Universität für Musik und darstellende Kunst Graz"
EU,23.91%,18. Platz,"277,190 €",Technische Universität Wien (14.7 Mio. €),"Technische Universität Wien, Universität Wien, Technische Universität Graz, Universität Innsbruck, Medizinische Universität Wien, Universität für Bodenkultur Wien, Montanuniversität Leoben, Medizinische Universität Innsbruck, Universität Salzburg, Universität Graz, Universität Linz, Medizinische Universität Graz, Wirtschaftsuniversität Wien, Universität Klagenfurt, Universität für Weiterbildung Krems, Veterinärmedizinische Universität Wien, Universität für angewandte Kunst Wien"


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
sonstige,19.24%,3. Platz,"2,475,309 €",Medizinische Universität Wien (33.2 Mio. €),"Medizinische Universität Wien, Universität Wien"
EU,16.09%,13. Platz,"2,070,273 €",Technische Universität Wien (14.7 Mio. €),"Technische Universität Wien, Universität Wien, Technische Universität Graz, Universität Innsbruck, Medizinische Universität Wien, Universität für Bodenkultur Wien, Montanuniversität Leoben, Medizinische Universität Innsbruck, Universität Salzburg, Universität Graz, Universität Linz, Medizinische Universität Graz"
"Private (Stiftungen, Vereine, etc.)",15.23%,6. Platz,"1,959,377 €",Medizinische Universität Wien (9.1 Mio. €),"Medizinische Universität Wien, Veterinärmedizinische Universität Wien, Universität für Bodenkultur Wien, Universität Wien, Universität Innsbruck"
FWF,11.28%,13. Platz,"1,450,782 €",Universität Wien (41.1 Mio. €),"Universität Wien, Technische Universität Wien, Medizinische Universität Wien, Universität Innsbruck, Universität Graz, Medizinische Universität Innsbruck, Universität für Bodenkultur Wien, Universität Linz, Universität Salzburg, Technische Universität Graz, Veterinärmedizinische Universität Wien, Medizinische Universität Graz"


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
sonstige,33.55%,1. Platz,"33,211,761 €",(diese),(keine)
Unternehmen,21.68%,4. Platz,"21,467,168 €",Medizinische Universität Graz (32.4 Mio. €),"Medizinische Universität Graz, Technische Universität Graz, Technische Universität Wien"


,Intern,Ranking,Erlöse,Beste Universität,Bessere Universitäten
WZW,,,,,
FWF,44.60%,17. Platz,"724,207 €",Universität Wien (41.1 Mio. €),"Universität Wien, Technische Universität Wien, Medizinische Universität Wien, Universität Innsbruck, Universität Graz, Medizinische Universität Innsbruck, Universität für Bodenkultur Wien, Universität Linz, Universität Salzburg, Technische Universität Graz, Veterinärmedizinische Universität Wien, Medizinische Universität Graz, Wirtschaftsuniversität Wien, Universität Klagenfurt, Montanuniversität Leoben, Universität für angewandte Kunst Wien"
"sonstige öffentlich-rechtliche Einrichtungen (Körperschaften, Stiftungen, Fonds etc.)",14.63%,17. Platz,"237,590 €",Technische Universität Graz (11.9 Mio. €),"Technische Universität Graz, Technische Universität Wien, Universität Innsbruck, Universität Linz, Montanuniversität Leoben, Universität für Bodenkultur Wien, Medizinische Universität Innsbruck, Universität Wien, Universität Salzburg, Universität Graz, Veterinärmedizinische Universität Wien, Wirtschaftsuniversität Wien, Medizinische Universität Wien, Medizinische Universität Graz, Universität Klagenfurt, Universität für Weiterbildung Krems"
